In [4]:
import numpy as np
from typing import List, Tuple
from ultralytics import YOLO

from deep_sort.application_util import preprocessing
from deep_sort.deep_sort import nn_matching
from deep_sort.deep_sort.detection import Detection
from deep_sort.deep_sort.tracker import Tracker
from deep_sort.tools import generate_detections as gdet


# own
from utils.utils import (get_frames, SEQ_01, SEQ_02, SEQ_03)
from utils.deepsort_utils import LABELS_DICT, UNKNOWN_DEFAULT

In [5]:
deepsort_model_ = './networks/mars-small128.pb' # TODO missing
yolo_model_ = '../../models/yolov8n.pt' # TODO missing

encoder = gdet.create_box_encoder(deepsort_model_, batch_size=1)
_metric = nn_matching.NearestNeighborDistanceMetric("cosine", 0.4, None)
tracker = Tracker(_metric, n_init=0)
detector = YOLO(yolo_model_)

In [18]:
class DeepSortObject():
    id: int
    label: str
    confidence: float
    xyxy: List[float]
    mask: np.ndarray

    @property
    def xywh(self):
        raise NotImplementedError

In [8]:
frames = [(_l, _r) for _l, _r in [get_frames(frame_num_=i, seq_dir_=SEQ_01) for i in range(2)]]

detections = []
for _i_frame, _frame in enumerate(frames):
    detector_pred = detector(_frame, classes=[0, 1, 2])
    bboxes = []
    yolobbox2objdata = {}
    boxes = detector_pred[0].boxes
    if not detector_pred:
        print(f"no detections in frame {_i_frame}")
        continue

    # for box in boxes:
    #     conf = float(box.conf[0])
    #     cls_val = int(box.cls[0])
    #     cls = LABELS_DICT.get(cls_val)
    #     # box.xyxy is tlbr (top-left, bottom-right)
    #     # convert from tensor to list
    #     box = box.xyxy.tolist()[0]
    #     yolobbox2objdata[tuple(box)] = (cls, conf)
    #     top_left_x, top_left_y = box[0], box[1]
    #     width = box[2] - box[0]
    #     height = box[3] - box[1]
    #     bbox = [top_left_x, top_left_y, width, height]
    #     bboxes.append(bbox)
    


0: 256x640 3 persons, 2 bicycles, 1: 256x640 3 persons, 2 bicycles, 32.7ms
Speed: 0.2ms preprocess, 16.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 256x640 3 persons, 3 bicycles, 1: 256x640 2 persons, 2 bicycles, 36.3ms
Speed: 0.2ms preprocess, 18.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)


In [16]:
_dets_i = detections[0]

_det_0 = _dets_i[0]

print(f"confidences: {_det_0.conf[0]}")
print(f"classes: {LABELS_DICT.get(int(_det_0.cls[0]))} ({_det_0.cls[0]})")



confidences: 0.8920082449913025
classes: Pedestrian (0.0)


In [10]:
_dets_i = detections[0]

objects_i = []
for _det in _dets_i:
    _dso = DeepSortObject()
    _dso.id = int(_det.cls[0])
    _dso.label = LABELS_DICT.get(int(_dso.id), UNKNOWN_DEFAULT)
    _dso.confidence = float(_det.conf[0])
    _dso.xyxy = _det.xyxy.tolist()[0]
    _dso.mask = None
    
    objects_i.append(_dso)

5
